## Camera Calibration

### Helper functions

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import os
import pickle
import matplotlib.image as mpimg

def store_img(img,dstfldr,filename,extra):
    if filename is '':
        return
    if dstfldr != '':
        dstfldr = "output_images" + '/' + dstfldr
    else:
        dstfldr = 'output_images'
    (head, tail) = os.path.split(filename)
    (root, ext) = os.path.splitext(tail)
    new_filename = os.path.join(dstfldr, root + extra + ".jpg")
    cv2.imwrite(new_filename, img)

def draw_region(img,lines, color=[0, 0, 255], thickness=2):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):

    return cv2.addWeighted(initial_img, α, img, β, γ)

# from https://stackoverflow.com/questions/42420470/opencv-subplots-images-with-titles-and-space-around-borders
def cvSubplot(imgs,     # 2d np array of imgs (each img an np arrays of depth 1 or 3).
              pad=50,   # number of pixels to use for padding between images. must be even
              titles=None,  # (optional) np array of subplot titles
              ):
    '''
    Makes cv2 based subplots. Useful to plot image in actual pixel size
    '''
    rows = np.shape(imgs)[0]
    cols = np.shape(imgs)[1]

    subplot_shapes = np.array([list(map(np.shape, x)) for x in imgs])
    if(subplot_shapes.shape[2] == 3):    
        sp_height, sp_width, depth = np.max(np.max(subplot_shapes, axis=0), axis=0)
    else:
        sp_height, sp_width = np.max(np.max(subplot_shapes, axis=0), axis=0)

    title_pad = 50
    if titles is not None:
        pad_top = pad + title_pad
    else:
        pad_top = pad

    if (subplot_shapes.shape[2] == 3):
        frame = np.ones((rows*(sp_height+pad_top), cols*(sp_width+pad), depth )) * 255
    else:
        frame = np.ones((rows*(sp_height+pad_top), cols*(sp_width+pad) )) * 255

    for r in range(rows):
        for c in range(cols):
            img = imgs[r, c]
            h = img.shape[0]
            w = img.shape[1]
            y0 = r * (sp_height+pad_top) + pad_top//2
            x0 = c * (sp_width+pad) + pad//2
            if (subplot_shapes.shape[2] == 3):
                frame[y0:y0+h, x0:x0+w, :] = img
            else:
                frame[y0:y0+h, x0:x0+w] = img

            if titles is not None:
                frame = cv2.putText(frame, titles[r, c], (x0 , y0-title_pad//4), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0),2)

    return frame
    
def undist(img, mtx, dist, filename = '',dstfldr = ''):
    
    img_size = (img.shape[1],img.shape[0])
    #print("image_size ",img_size)
    # Distortion Correction
    undist = cv2.undistort(img,mtx,dist,None,mtx)
    imgs = np.array([[img,undist]])
    ttls = np.array([['original','undistorted']])
    store_img(cvSubplot(imgs,titles=ttls),dstfldr,filename,"_undistorted")   
    
    return undist 

def unwarp(img, src, dst, filename = '',dstfldr=''):
    
    img_size = (img.shape[1],img.shape[0])
    
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    
    imgs = np.array([[img,warped]])
    ttls = np.array([['original','warped']])
    store_img(cvSubplot(imgs,titles=ttls),dstfldr,filename,"_warped")  
    
    return warped, M, Minv


def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0,255)):
    
    # Apply the following steps to img
    # 1) the input image must be in grayscale
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    # 3) Take the absolute value of the derivative or gradient
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    # 5) Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    # 6) Return this mask as your binary_output image
    binary_output = np.copy(img) # Remove this line
    if ((orient is not 'x') and (orient is not 'y')):
        return None
    thresh_min = thresh[0]
    thresh_max = thresh[1]
    gray = np.copy(img)
    sobelx = cv2.Sobel(gray,cv2.CV_64F,(0,1)[orient == 'x'],(0,1)[orient == 'y'],ksize=sobel_kernel)
    abs_sobel = np.absolute(sobelx)
    abs_sobel_scale = np.uint8(255*abs_sobel/np.max(abs_sobel))
    binary_output = np.zeros_like(abs_sobel_scale)
    binary_output[(abs_sobel_scale >= thresh_min) & (abs_sobel_scale <= thresh_max)] = 1

    return binary_output

def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    
    # Apply the following steps to img
    # 1) the input image must be in grayscale
    # 2) Take the gradient in x and y separately
    # 3) Calculate the magnitude 
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    # 5) Create a binary mask where mag thresholds are met
    # 6) Return this mask as your binary_output image
    gray = np.copy(img)
    sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize=sobel_kernel)
    sobely = cv2.Sobel(gray,cv2.CV_64F,0,1,ksize=sobel_kernel)
    mag_sobel = np.sqrt(np.square(sobelx)+np.square(sobely))
    mag_scaled = np.uint8(255*mag_sobel/np.max(mag_sobel))
    binary_output = np.zeros_like(mag_scaled)
    binary_output[(mag_scaled >= mag_thresh[0]) & (mag_scaled <= mag_thresh[1])] = 1

    return binary_output

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    
    # Apply the following steps to img
    # 1) the input image must be in grayscale
    # 2) Take the gradient in x and y separately
    # 3) Take the absolute value of the x and y gradients
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    # 5) Create a binary mask where direction thresholds are met
    # 6) Return this mask as your binary_output image
    gray = np.copy(img)
    abs_sobelx = np.absolute(cv2.Sobel(gray,cv2.CV_64F,1,0,ksize=sobel_kernel))
    abs_sobely = np.absolute(cv2.Sobel(gray,cv2.CV_64F,0,1,ksize=sobel_kernel))
    direction = np.arctan2(abs_sobely,abs_sobelx)
    binary_output = np.zeros_like(gray)
    binary_output[(direction >= thresh[0]) & (direction <= thresh[1])] = 1
    return binary_output

def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    
    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### Find the four below boundaries of the window ###
        win_xleft_low = leftx_current  - margin 
        win_xleft_high = leftx_current  + margin  
        win_xright_low = rightx_current - margin  
        win_xright_high = rightx_current + margin  
        
        # Draw the windows on the visualization image
        #cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 5) 
        #cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 5) 
        
        ### Identify the nonzero pixels in x and y within the window ###
        #twin = binary_warped[win_xleft_low:win_xleft_high,win_y_low:win_y_high].nonzero()
        good_left_inds = ((nonzerox >= win_xleft_low) & (nonzerox <= win_xleft_high) & (nonzeroy >= win_y_low) & (nonzeroy <= win_y_high)).nonzero()[0]
        #print(type(twin[1]))
        good_right_inds = ((nonzerox >= win_xright_low) & (nonzerox <= win_xright_high) & (nonzeroy >= win_y_low) & (nonzeroy <= win_y_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        ###  If you found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        
        if len(good_left_inds) > minpix:
            leftx_current =  np.int(np.mean(nonzerox[good_left_inds], axis=0))
            #print(leftx_current)
        if len(good_right_inds) > minpix:
            rightx_current =  np.int(np.mean(nonzerox[good_right_inds], axis=0))
            #print(rightx_current)

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        print("AN ERROR",left_lane_inds)
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    ###  Fit a second order polynomial to each using `np.polyfit` ###
    left_fit = np.polyfit(lefty,leftx,2)
    right_fit = np.polyfit(righty,rightx,2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )#list(range(0, binary_warped.shape[0]))
    try:
        left_fitx = np.polyval(left_fit,ploty) #left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = np.polyval(right_fit,ploty) #right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    left_lines = []
    right_lines = []
    for idx in range(len(ploty[:-1])):
        left_lines.append((int(left_fitx[idx]),int(ploty[idx]),int(left_fitx[idx+1]),int(ploty[idx+1])))
        right_lines.append((int(right_fitx[idx]),int(ploty[idx]),int(right_fitx[idx+1]),int(ploty[idx+1])))
    lines = np.array([left_lines,right_lines])
    draw_region(out_img,lines,[0, 255, 0])

    return out_img,left_fit,right_fit,ploty,left_fitx,right_fitx

### Getting the object and image points

In [2]:


nx = 9
ny = 6
camera_image_size = cv2.cvtColor(cv2.imread("test_images/test1.jpg"),cv2.COLOR_BGR2GRAY).shape
#print(camera_image_size)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((nx*ny,3), np.float32)
objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
calib_images = glob.glob('camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in calib_images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img_corners = cv2.drawChessboardCorners(img, (nx,ny), corners, ret)
        store_img(img_corners,"CameraCalibration/Corners",fname,"_corners")
points_pickle = {"objpoints":objpoints,"imgpoints":imgpoints}
with open('ObjImagePoints_pickle.pickle', 'wb') as f:
    pickle.dump(points_pickle, f)


### Calculating the camera matrix and distortion coefficients

In [3]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints,imgpoints,(camera_image_size[1],camera_image_size[0]),None,None)

dist_pickle = {"camMtrx":mtx,"distCoe":dist}
with open('CameraMatrix_DistrotionCoefficients.pickle', 'wb') as f:
    pickle.dump(dist_pickle, f)

for index,fname in enumerate(calib_images):
    undist_image = undist(cv2.imread(fname),mtx,dist,fname,"CameraCalibration/Undistorted")
    # Convert to gray-scale
    gray = cv2.cvtColor(undist_image, cv2.COLOR_BGR2GRAY)
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
    if (ret == True):
        cv2.drawChessboardCorners(undist_image, (nx, ny), corners, ret)
        #print("CORNERS ",corners)
        src = np.float32([corners[0],corners[nx-1],corners[((ny-1)*nx)],corners[(nx*ny)-1]])
        #print("SRC",src)
        dst = np.float32([[100,100],[1200,100],[100,600],[1200,600]])
        warped_image,M,Minv = unwarp(undist_image,src,dst,fname,"CameraCalibration/Warped")

### Applying the distortion correction to the test images

In [4]:

dist_pickle = pickle.load( open( "CameraMatrix_DistrotionCoefficients.pickle", "rb" ) )
p_mtx = dist_pickle["camMtrx"]
p_dist = dist_pickle["distCoe"]

sobel_kernel_size=3

s_thresh=(170, 255)
sx_thresh=(20, 100)
sm_thresh=(30, 100)
sd_thresh=(0.7, 1.3)

#imshape = camera_image_size
#region_top_length = 1/16
#region_height = 0.4
#region_bottom_str = 0.12
#region_bottom_end = 0.041

x1 = 180#int(region_bottom_str * imshape[1])
y1 = 720#imshape[0]
x2 = 592#int(imshape[1]*(1-region_top_length)/2)
y2 = 450#int(imshape[0]*(1-region_height))
x3 = 690#int(imshape[1]*(1+region_top_length)/2)
y3 = 450#int(imshape[0]*(1-region_height))
x4 = 1150#int(imshape[1]*(1 - region_bottom_end))
y4 = 720#imshape[0]

x1_des = 300
y1_des = 720
x2_des = 300
y2_des = 0
x3_des = 1000
y3_des = 0
x4_des = 1000
y4_des = 720

def advanced_pipeline(img,filename=''):
    '''
    Distortion Correction
    '''
    undist_image = undist(img,p_mtx,p_dist)
    
    undist_original_subplot = cvSubplot(np.array([[img,undist_image]]),
                                       titles=np.array([["Original",'Undistorted']]))
    store_img(undist_original_subplot,'',filename,"_stage1_undistorted")
    
    '''
    Color conversion
    '''
    # Convert to HLS color space and separate the L and S channels
    hls = cv2.cvtColor(undist_image, cv2.COLOR_BGR2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    #plt.imshow(l_channel, cmap='gray')
    #plt.show()
    #plt.imshow(s_channel, cmap='gray')
    #plt.show()
    #print(l_channel.shape)
    
    '''
    Gradient Threshold
    '''
    # Gradient threshold
    gradx = abs_sobel_thresh(l_channel, orient='x', sobel_kernel=sobel_kernel_size, thresh=sx_thresh)
    grady = abs_sobel_thresh(l_channel, orient='y', sobel_kernel=sobel_kernel_size, thresh=sx_thresh)
    mag_binary = mag_thresh(l_channel, sobel_kernel=sobel_kernel_size, mag_thresh=sm_thresh)
    dir_binary = dir_threshold(l_channel, sobel_kernel=sobel_kernel_size, thresh=sd_thresh)
    '''
    Combining the the gradient threshold
    '''
    grad_combined = np.zeros_like(dir_binary)
    grad_combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    #print(grad_combined.shape)
    #plt.imshow(grad_combined, cmap='gray')
    #plt.show()
    store_img(grad_combined*255,'',filename,"_stage2_gradThrsh")
    
    '''
    Color Threshold
    '''
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    #plt.imshow(s_binary, cmap='gray')
    #plt.show()
    store_img(s_binary*255,'',filename,"_stage3_clrThrsh")
    
    '''
    Combining the color threshold with the gradient threshold
    '''
    # Stack each channel
    color_binary = np.dstack(( s_binary, grad_combined, np.zeros_like(grad_combined))) * 255
    
    store_img(color_binary,'',filename,"_stage4_clrGradThrsh")
    
    binary_mask = np.zeros_like(s_binary)
    binary_mask[(s_binary == 1) | (grad_combined == 1)]=1
    #plt.imshow(binary_mask, cmap='gray')
    #plt.show()
    store_img(binary_mask*255,'',filename,"_stage5_binary_combo")
    
    '''
    Warp
    '''
    region_lines = np.array([[(x1,y1,x2,y2),(x2,y2,x3,y3),(x3,y3,x4,y4),(x4,y4,x1,y1)]])
    region_line_img = np.zeros((undist_image.shape[0], undist_image.shape[1], 3), dtype=np.uint8)
    draw_region(region_line_img,region_lines)
    region_image = weighted_img(region_line_img,undist_image)
    
    #plt.imshow(cv2.cvtColor(region_image,cv2.COLOR_BGR2RGB))
    #plt.show()
    
    src = np.float32([[x1,y1],[x2,y2],[x3,y3],[x4,y4]])
    dst = np.float32([[x1_des,y1_des],[x2_des,y2_des],[x3_des,y3_des],[x4_des,y4_des]])
    warped, M, Minv = unwarp(undist_image, src, dst)#, filename = '',dstfldr=''
    
    region_lines = np.array([[(x1_des,y1_des,x2_des,y2_des),
                              (x2_des,y2_des,x3_des,y3_des),
                              (x3_des,y3_des,x4_des,y4_des),
                              (x4_des,y4_des,x1_des,y1_des)]])
    region_line_img = np.zeros((warped.shape[0], warped.shape[1], 3), dtype=np.uint8)
    draw_region(region_line_img,region_lines)
    warped_region_image = weighted_img(region_line_img,warped)
    
    #plt.imshow(cv2.cvtColor(warped_region_image,cv2.COLOR_BGR2RGB))
    #plt.show()
    
    warped_original_subplot = cvSubplot(np.array([[region_image,warped_region_image]]),
                                       titles=np.array([["Undistorted with src points",'warped with the dest. points']]))
    store_img(warped_original_subplot,'',filename,"_stage6_warped_src_dest")
    
    warped_binary, M, Minv = unwarp(binary_mask, src, dst)#, filename = '',dstfldr=''
    #plt.imshow(warped_binary, cmap='gray')
    #plt.show()
    store_img(warped_binary*255,'',filename,"_stage7_warped_binary")
    
    '''
    Lane line detection and curve fitting 
    '''
    fit_poly,left_fit,right_fit,ploty,left_fitx,right_fitx = fit_polynomial(warped_binary)
    #print(left_fit)
    #print(right_fit)
    #plt.imshow(fit_poly)
    #plt.show()
    store_img(fit_poly,'',filename,"_stage8_fit_curve")
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped_binary).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    # Combine the result with the original image
    img_lane_bndry = cv2.addWeighted(undist_image, 1, newwarp, 0.3, 0)
    store_img(img_lane_bndry,'',filename,"_stage9_lane_boundary")
    #plt.imshow(cv2.cvtColor(img_lane_bndry,cv2.COLOR_BGR2RGB))
    #plt.show()
    
    '''
    curvature calculation
    '''
    
    out_image = img
    return out_image

test_images = os.listdir("test_images/")
for img in test_images:
    img = "test_images/" + img
    print(img)
    out_image = advanced_pipeline(cv2.imread(img),img)
    

test_images/straight_lines1.jpg
test_images/straight_lines2.jpg
test_images/test1.jpg
test_images/test2.jpg
test_images/test3.jpg
test_images/test4.jpg
test_images/test5.jpg
test_images/test6.jpg
